In [1]:
import pandas as pd
import os
import numpy as np

In [2]:
def process_df(path):
    df=pd.read_csv(path)
#     df=df[df.Metric=='f1-score']
    df['MID']=df['MID'].apply(lambda x: x.split('_')[1] if '_' in x else x)
    df['MID_Num']=df['MID'].apply(lambda x: x[1:]).astype(int)
    df=df[['MID','MID_Num','Classifier','Metric','Seizure']].sort_values('MID_Num').reset_index(drop=True).round(4)
    df=df.pivot(index=['Classifier', 'Metric'], 
                      columns='MID_Num', 
                      values='Seizure')
    
    # Reset index to make Classifier and Metric regular columns
    df_pivoted = df.reset_index()

    # Optional: rename the columns index name to None for cleaner output
    df_pivoted.columns.name = None

    # Define the desired order of metrics
    metric_order = ['precision', 'recall', 'f1-score', 'support']

    # Reorder using categorical type
    df_pivoted['Metric'] = pd.Categorical(df_pivoted['Metric'], 
                                           categories=metric_order, 
                                           ordered=True)

    # Sort by Classifier and then by Metric
    df_pivoted = df_pivoted.sort_values(['Classifier', 'Metric']).reset_index(drop=True)

    return df_pivoted

In [3]:
#baseline results
base_path='./results_csvs/'

base_files=[os.path.join(base_path,x) for x in os.listdir(base_path) if 'csv' in x]

all_files=base_files+[
    '../MLMethods/Logistic Regression_results.csv',
    '../MLMethods/K-Nearest Neighbors_results.csv',
    '../MLMethods/Decision Tree_results.csv',
    '../MLMethods/Random Forest_results.csv',
    '../DETRtime/DETRtime_results_v1.csv',
    './augunet1D_results_v1.csv'
]

full_results=pd.DataFrame()
for file in all_files:
    df=process_df(file)
    full_results=pd.concat([full_results, df])

full_results.columns=full_results.columns[:2].tolist()+[f'M{x}' for x in full_results.columns[2:]]
classifier_order=['Logistic Regression','K-Nearest Neighbors', 'Decision Tree', 'Random Forest',
'CNN', 'PyramidalCNN',
'LSTM','biLSTM','ConvLSTM',
'EEGNet','InceptionTime','Xception','UNet','SalientSleepNet','DETRtime','AugUNet1D (ours)']
# full_results=full_results.set_index(['Classifier', 'Metric'])
# Create a categorical type with the custom order
full_results['Classifier'] = pd.Categorical(full_results['Classifier'], 
                                   categories=classifier_order, 
                                   ordered=True)
full_results = full_results.sort_values(['Classifier', 'Metric']).reset_index(drop=True)
full_results = full_results[full_results.Metric!='support']
full_results['avg']=full_results[[x for x in full_results.columns[2:]]].mean(axis=1)
full_results = full_results.round(4)
full_results.to_csv('full_results_v1.csv',index=False)
full_results

,Classifier,Metric,M1,M2,M3,M4,M5,M6,M7,M8,M9,M10,avg
0,Logistic Regression,precision,0.0867,0.0902,0.0994,0.0869,0.1105,0.1028,0.1249,0.0686,0.0657,0.0325,0.0868
1,Logistic Regression,recall,0.9759,0.9605,0.9662,0.9818,0.9719,0.9831,0.9749,0.9802,0.9758,0.9888,0.9759
2,Logistic Regression,f1-score,0.1593,0.1650,0.1802,0.1597,0.1984,0.1861,0.2214,0.1282,0.1231,0.0629,0.1584
4,K-Nearest Neighbors,precision,0.1529,0.1935,0.1442,0.1490,0.2592,0.1678,0.2493,0.1306,0.1014,0.0570,0.1605
5,K-Nearest Neighbors,recall,0.7030,0.8340,0.6582,0.7909,0.8193,0.7511,0.7113,0.8861,0.7585,0.8324,0.7745
6,K-Nearest Neighbors,f1-score,0.2512,0.3142,0.2365,0.2507,0.3938,0.2743,0.3692,0.2276,0.1788,0.1067,0.2603
8,Decision Tree,precision,0.2650,0.2241,0.2410,0.2573,0.3417,0.2567,0.3600,0.1758,0.1430,0.0926,0.2357
9,Decision Tree,recall,0.5566,0.6616,0.4515,0.6045,0.5462,0.6076,0.6402,0.7475,0.6329,0.5866,0.6035
10,Decision Tree,f1-score,0.3590,0.3348,0.3142,0.3609,0.4204,0.3609,0.4608,0.2846,0.2333,0.1599,0.3289
12,Random Forest,precision,0.2701,0.2193,0.2299,0.2922,0.4574,0.1984,0.3553,0.1950,0.1176,0.0849,0.2420


In [4]:
#baseline results
base_path='./results_csvs_v2/'

base_files=[os.path.join(base_path,x) for x in os.listdir(base_path) if 'csv' in x]

all_files=base_files+[
    '../MLMethods/Logistic Regression_results_v2.csv',
    '../MLMethods/K-Nearest Neighbors_results_v2.csv',
    '../MLMethods/Decision Tree_results_v2.csv',
    '../MLMethods/Random Forest_results_v2.csv',
    '../DETRtime/DETRtime_results.csv',
    './augunet1D_results_v2.csv'
]

full_results=pd.DataFrame()
for file in all_files:
    df=process_df(file)
    full_results=pd.concat([full_results, df])

full_results.columns=full_results.columns[:2].tolist()+[f'M{x}' for x in full_results.columns[2:]]
classifier_order=['Logistic Regression','K-Nearest Neighbors', 'Decision Tree', 'Random Forest',
'CNN', 'PyramidalCNN',
'LSTM','biLSTM','ConvLSTM',
'EEGNet','InceptionTime','Xception','UNet','SalientSleepNet','DETRtime','AugUNet1D (ours)']
# full_results=full_results.set_index(['Classifier', 'Metric'])
# Create a categorical type with the custom order
full_results['Classifier'] = pd.Categorical(full_results['Classifier'], 
                                   categories=classifier_order, 
                                   ordered=True)
full_results = full_results.sort_values(['Classifier', 'Metric']).reset_index(drop=True)
full_results = full_results[full_results.Metric!='support']
full_results['avg']=full_results[[x for x in full_results.columns[2:]]].mean(axis=1)
full_results = full_results.round(4)
full_results.to_csv('full_results_v2.csv',index=False)
full_results

,Classifier,Metric,M1,M2,M3,M4,M5,M6,M7,M8,M9,M10,avg
0,Logistic Regression,precision,0.0867,0.0902,0.0994,0.0869,0.1105,0.1028,0.1249,0.0686,0.0657,0.0325,0.0868
1,Logistic Regression,recall,0.9759,0.9605,0.9662,0.9818,0.9719,0.9831,0.9749,0.9802,0.9758,0.9888,0.9759
2,Logistic Regression,f1-score,0.1593,0.1650,0.1802,0.1597,0.1984,0.1861,0.2214,0.1282,0.1231,0.0629,0.1584
4,K-Nearest Neighbors,precision,0.1529,0.1935,0.1442,0.1490,0.2592,0.1678,0.2493,0.1306,0.1014,0.0570,0.1605
5,K-Nearest Neighbors,recall,0.7030,0.8340,0.6582,0.7909,0.8193,0.7511,0.7113,0.8861,0.7585,0.8324,0.7745
6,K-Nearest Neighbors,f1-score,0.2512,0.3142,0.2365,0.2507,0.3938,0.2743,0.3692,0.2276,0.1788,0.1067,0.2603
8,Decision Tree,precision,0.2669,0.2152,0.2530,0.2524,0.3333,0.2500,0.3605,0.1768,0.1370,0.0922,0.2337
9,Decision Tree,recall,0.5591,0.6624,0.4515,0.6045,0.5542,0.5992,0.6485,0.7475,0.6184,0.5922,0.6038
10,Decision Tree,f1-score,0.3613,0.3249,0.3242,0.3561,0.4163,0.3528,0.4634,0.2860,0.2244,0.1595,0.3269
12,Random Forest,precision,0.2926,0.2309,0.2490,0.2899,0.4845,0.2094,0.3624,0.2017,0.1399,0.0949,0.2555


In [5]:
#baseline results
base_path='./results_csvs_v3/'

base_files=[os.path.join(base_path,x) for x in os.listdir(base_path) if 'csv' in x]

all_files=base_files+[
    '../MLMethods/Logistic Regression_results_v3.csv',
    '../MLMethods/K-Nearest Neighbors_results_v3.csv',
    '../MLMethods/Decision Tree_results_v3.csv',
    '../MLMethods/Random Forest_results_v3.csv',
    '../DETRtime/DETRtime_results_v3.csv',
    './augunet1D_results_v3.csv'
]

full_results=pd.DataFrame()
for file in all_files:
    df=process_df(file)
    full_results=pd.concat([full_results, df])

full_results.columns=full_results.columns[:2].tolist()+[f'M{x}' for x in full_results.columns[2:]]
classifier_order=['Logistic Regression','K-Nearest Neighbors', 'Decision Tree', 'Random Forest',
'CNN', 'PyramidalCNN',
'LSTM','biLSTM','ConvLSTM',
'EEGNet','InceptionTime','Xception','UNet','SalientSleepNet','DETRtime','AugUNet1D (ours)']
# full_results=full_results.set_index(['Classifier', 'Metric'])
# Create a categorical type with the custom order
full_results['Classifier'] = pd.Categorical(full_results['Classifier'], 
                                   categories=classifier_order, 
                                   ordered=True)
full_results = full_results.sort_values(['Classifier', 'Metric']).reset_index(drop=True)
full_results = full_results[full_results.Metric!='support']
full_results['avg']=full_results[[x for x in full_results.columns[2:]]].mean(axis=1)
full_results = full_results.round(4)
full_results.to_csv('full_results_v3.csv',index=False)
full_results

,Classifier,Metric,M1,M2,M3,M4,M5,M6,M7,M8,M9,M10,avg
0,Logistic Regression,precision,0.0867,0.0902,0.0994,0.0869,0.1105,0.1028,0.1249,0.0686,0.0657,0.0325,0.0868
1,Logistic Regression,recall,0.9759,0.9605,0.9662,0.9818,0.9719,0.9831,0.9749,0.9802,0.9758,0.9888,0.9759
2,Logistic Regression,f1-score,0.1593,0.1650,0.1802,0.1597,0.1984,0.1861,0.2214,0.1282,0.1231,0.0629,0.1584
4,K-Nearest Neighbors,precision,0.1529,0.1935,0.1442,0.1490,0.2592,0.1678,0.2493,0.1306,0.1014,0.0570,0.1605
5,K-Nearest Neighbors,recall,0.7030,0.8340,0.6582,0.7909,0.8193,0.7511,0.7113,0.8861,0.7585,0.8324,0.7745
6,K-Nearest Neighbors,f1-score,0.2512,0.3142,0.2365,0.2507,0.3938,0.2743,0.3692,0.2276,0.1788,0.1067,0.2603
8,Decision Tree,precision,0.2706,0.2184,0.2442,0.2573,0.3310,0.2540,0.3581,0.1782,0.1421,0.0910,0.2345
9,Decision Tree,recall,0.5624,0.6616,0.4430,0.6045,0.5622,0.6034,0.6444,0.7673,0.6232,0.5866,0.6059
10,Decision Tree,f1-score,0.3654,0.3283,0.3148,0.3609,0.4167,0.3575,0.4604,0.2892,0.2314,0.1575,0.3282
12,Random Forest,precision,0.2811,0.2247,0.2290,0.2801,0.4742,0.2060,0.3565,0.1978,0.1182,0.0847,0.2452


In [6]:
#baseline results
base_path='./results_csvs_v4/'

base_files=[os.path.join(base_path,x) for x in os.listdir(base_path) if 'csv' in x]

all_files=base_files+[
    '../MLMethods/Logistic Regression_results_v3.csv',
    '../MLMethods/K-Nearest Neighbors_results_v3.csv',
    '../MLMethods/Decision Tree_results_v3.csv',
    '../MLMethods/Random Forest_results_v3.csv',
    '../DETRtime/DETRtime_results_v3.csv',
    './augunet1D_results_v3.csv'
]

full_results=pd.DataFrame()
for file in all_files:
    df=process_df(file)
    full_results=pd.concat([full_results, df])

full_results.columns=full_results.columns[:2].tolist()+[f'M{x}' for x in full_results.columns[2:]]
classifier_order=['Logistic Regression','K-Nearest Neighbors', 'Decision Tree', 'Random Forest',
'CNN', 'PyramidalCNN',
'LSTM','biLSTM','ConvLSTM',
'EEGNet','InceptionTime','Xception','UNet','SalientSleepNet','DETRtime','AugUNet1D (ours)']
# full_results=full_results.set_index(['Classifier', 'Metric'])
# Create a categorical type with the custom order
full_results['Classifier'] = pd.Categorical(full_results['Classifier'], 
                                   categories=classifier_order, 
                                   ordered=True)
full_results = full_results.sort_values(['Classifier', 'Metric']).reset_index(drop=True)
full_results = full_results[full_results.Metric!='support']
full_results['avg']=full_results[[x for x in full_results.columns[2:]]].mean(axis=1)
full_results = full_results.round(4)
full_results.to_csv('full_results_v4.csv',index=False)
full_results

,Classifier,Metric,M1,M2,M3,M4,M5,M6,M7,M8,M9,M10,avg
0,Logistic Regression,precision,0.0867,0.0902,0.0994,0.0869,0.1105,0.1028,0.1249,0.0686,0.0657,0.0325,0.0868
1,Logistic Regression,recall,0.9759,0.9605,0.9662,0.9818,0.9719,0.9831,0.9749,0.9802,0.9758,0.9888,0.9759
2,Logistic Regression,f1-score,0.1593,0.1650,0.1802,0.1597,0.1984,0.1861,0.2214,0.1282,0.1231,0.0629,0.1584
4,K-Nearest Neighbors,precision,0.1529,0.1935,0.1442,0.1490,0.2592,0.1678,0.2493,0.1306,0.1014,0.0570,0.1605
5,K-Nearest Neighbors,recall,0.7030,0.8340,0.6582,0.7909,0.8193,0.7511,0.7113,0.8861,0.7585,0.8324,0.7745
6,K-Nearest Neighbors,f1-score,0.2512,0.3142,0.2365,0.2507,0.3938,0.2743,0.3692,0.2276,0.1788,0.1067,0.2603
8,Decision Tree,precision,0.2706,0.2184,0.2442,0.2573,0.3310,0.2540,0.3581,0.1782,0.1421,0.0910,0.2345
9,Decision Tree,recall,0.5624,0.6616,0.4430,0.6045,0.5622,0.6034,0.6444,0.7673,0.6232,0.5866,0.6059
10,Decision Tree,f1-score,0.3654,0.3283,0.3148,0.3609,0.4167,0.3575,0.4604,0.2892,0.2314,0.1575,0.3282
12,Random Forest,precision,0.2811,0.2247,0.2290,0.2801,0.4742,0.2060,0.3565,0.1978,0.1182,0.0847,0.2452


In [7]:
v1=pd.read_csv('./full_results_v1.csv')
v2=pd.read_csv('./full_results_v2.csv')
v3=pd.read_csv('./full_results_v3.csv')
v4=pd.read_csv('./full_results_v4.csv')

In [8]:
v1

,Classifier,Metric,M1,M2,M3,M4,M5,M6,M7,M8,M9,M10,avg
0,Logistic Regression,precision,0.0867,0.0902,0.0994,0.0869,0.1105,0.1028,0.1249,0.0686,0.0657,0.0325,0.0868
1,Logistic Regression,recall,0.9759,0.9605,0.9662,0.9818,0.9719,0.9831,0.9749,0.9802,0.9758,0.9888,0.9759
2,Logistic Regression,f1-score,0.1593,0.1650,0.1802,0.1597,0.1984,0.1861,0.2214,0.1282,0.1231,0.0629,0.1584
3,K-Nearest Neighbors,precision,0.1529,0.1935,0.1442,0.1490,0.2592,0.1678,0.2493,0.1306,0.1014,0.0570,0.1605
4,K-Nearest Neighbors,recall,0.7030,0.8340,0.6582,0.7909,0.8193,0.7511,0.7113,0.8861,0.7585,0.8324,0.7745
5,K-Nearest Neighbors,f1-score,0.2512,0.3142,0.2365,0.2507,0.3938,0.2743,0.3692,0.2276,0.1788,0.1067,0.2603
6,Decision Tree,precision,0.2650,0.2241,0.2410,0.2573,0.3417,0.2567,0.3600,0.1758,0.1430,0.0926,0.2357
7,Decision Tree,recall,0.5566,0.6616,0.4515,0.6045,0.5462,0.6076,0.6402,0.7475,0.6329,0.5866,0.6035
8,Decision Tree,f1-score,0.3590,0.3348,0.3142,0.3609,0.4204,0.3609,0.4608,0.2846,0.2333,0.1599,0.3289
9,Random Forest,precision,0.2701,0.2193,0.2299,0.2922,0.4574,0.1984,0.3553,0.1950,0.1176,0.0849,0.2420


In [9]:
v2

,Classifier,Metric,M1,M2,M3,M4,M5,M6,M7,M8,M9,M10,avg
0,Logistic Regression,precision,0.0867,0.0902,0.0994,0.0869,0.1105,0.1028,0.1249,0.0686,0.0657,0.0325,0.0868
1,Logistic Regression,recall,0.9759,0.9605,0.9662,0.9818,0.9719,0.9831,0.9749,0.9802,0.9758,0.9888,0.9759
2,Logistic Regression,f1-score,0.1593,0.1650,0.1802,0.1597,0.1984,0.1861,0.2214,0.1282,0.1231,0.0629,0.1584
3,K-Nearest Neighbors,precision,0.1529,0.1935,0.1442,0.1490,0.2592,0.1678,0.2493,0.1306,0.1014,0.0570,0.1605
4,K-Nearest Neighbors,recall,0.7030,0.8340,0.6582,0.7909,0.8193,0.7511,0.7113,0.8861,0.7585,0.8324,0.7745
5,K-Nearest Neighbors,f1-score,0.2512,0.3142,0.2365,0.2507,0.3938,0.2743,0.3692,0.2276,0.1788,0.1067,0.2603
6,Decision Tree,precision,0.2669,0.2152,0.2530,0.2524,0.3333,0.2500,0.3605,0.1768,0.1370,0.0922,0.2337
7,Decision Tree,recall,0.5591,0.6624,0.4515,0.6045,0.5542,0.5992,0.6485,0.7475,0.6184,0.5922,0.6038
8,Decision Tree,f1-score,0.3613,0.3249,0.3242,0.3561,0.4163,0.3528,0.4634,0.2860,0.2244,0.1595,0.3269
9,Random Forest,precision,0.2926,0.2309,0.2490,0.2899,0.4845,0.2094,0.3624,0.2017,0.1399,0.0949,0.2555


In [10]:
v3

,Classifier,Metric,M1,M2,M3,M4,M5,M6,M7,M8,M9,M10,avg
0,Logistic Regression,precision,0.0867,0.0902,0.0994,0.0869,0.1105,0.1028,0.1249,0.0686,0.0657,0.0325,0.0868
1,Logistic Regression,recall,0.9759,0.9605,0.9662,0.9818,0.9719,0.9831,0.9749,0.9802,0.9758,0.9888,0.9759
2,Logistic Regression,f1-score,0.1593,0.1650,0.1802,0.1597,0.1984,0.1861,0.2214,0.1282,0.1231,0.0629,0.1584
3,K-Nearest Neighbors,precision,0.1529,0.1935,0.1442,0.1490,0.2592,0.1678,0.2493,0.1306,0.1014,0.0570,0.1605
4,K-Nearest Neighbors,recall,0.7030,0.8340,0.6582,0.7909,0.8193,0.7511,0.7113,0.8861,0.7585,0.8324,0.7745
5,K-Nearest Neighbors,f1-score,0.2512,0.3142,0.2365,0.2507,0.3938,0.2743,0.3692,0.2276,0.1788,0.1067,0.2603
6,Decision Tree,precision,0.2706,0.2184,0.2442,0.2573,0.3310,0.2540,0.3581,0.1782,0.1421,0.0910,0.2345
7,Decision Tree,recall,0.5624,0.6616,0.4430,0.6045,0.5622,0.6034,0.6444,0.7673,0.6232,0.5866,0.6059
8,Decision Tree,f1-score,0.3654,0.3283,0.3148,0.3609,0.4167,0.3575,0.4604,0.2892,0.2314,0.1575,0.3282
9,Random Forest,precision,0.2811,0.2247,0.2290,0.2801,0.4742,0.2060,0.3565,0.1978,0.1182,0.0847,0.2452


In [11]:
v4

,Classifier,Metric,M1,M2,M3,M4,M5,M6,M7,M8,M9,M10,avg
0,Logistic Regression,precision,0.0867,0.0902,0.0994,0.0869,0.1105,0.1028,0.1249,0.0686,0.0657,0.0325,0.0868
1,Logistic Regression,recall,0.9759,0.9605,0.9662,0.9818,0.9719,0.9831,0.9749,0.9802,0.9758,0.9888,0.9759
2,Logistic Regression,f1-score,0.1593,0.1650,0.1802,0.1597,0.1984,0.1861,0.2214,0.1282,0.1231,0.0629,0.1584
3,K-Nearest Neighbors,precision,0.1529,0.1935,0.1442,0.1490,0.2592,0.1678,0.2493,0.1306,0.1014,0.0570,0.1605
4,K-Nearest Neighbors,recall,0.7030,0.8340,0.6582,0.7909,0.8193,0.7511,0.7113,0.8861,0.7585,0.8324,0.7745
5,K-Nearest Neighbors,f1-score,0.2512,0.3142,0.2365,0.2507,0.3938,0.2743,0.3692,0.2276,0.1788,0.1067,0.2603
6,Decision Tree,precision,0.2706,0.2184,0.2442,0.2573,0.3310,0.2540,0.3581,0.1782,0.1421,0.0910,0.2345
7,Decision Tree,recall,0.5624,0.6616,0.4430,0.6045,0.5622,0.6034,0.6444,0.7673,0.6232,0.5866,0.6059
8,Decision Tree,f1-score,0.3654,0.3283,0.3148,0.3609,0.4167,0.3575,0.4604,0.2892,0.2314,0.1575,0.3282
9,Random Forest,precision,0.2811,0.2247,0.2290,0.2801,0.4742,0.2060,0.3565,0.1978,0.1182,0.0847,0.2452


In [12]:
v1=v1[v1.Metric=='f1-score'][['Classifier','avg']]
v2=v2[v2.Metric=='f1-score'][['Classifier','avg']]
v3=v3[v3.Metric=='f1-score'][['Classifier','avg']]
v4=v4[v4.Metric=='f1-score'][['Classifier','avg']]

In [13]:
v_all=v1.rename(columns={'avg':'avg_f1_run1'}).merge(v2.rename(columns={'avg':'avg_f1_run2'}),on='Classifier')
v_all

,Classifier,avg_f1_run1,avg_f1_run2
0,Logistic Regression,0.1584,0.1584
1,K-Nearest Neighbors,0.2603,0.2603
2,Decision Tree,0.3289,0.3269
3,Random Forest,0.2820,0.2991
4,CNN,0.4976,0.4343
5,PyramidalCNN,0.5322,0.4957
6,LSTM,0.0000,0.0000
7,biLSTM,0.0000,0.0000
8,ConvLSTM,0.4994,0.4798
9,EEGNet,0.2652,0.0736


In [14]:
v_all=v_all.merge(v3.rename(columns={'avg':'avg_f1_run3'}),on='Classifier')
v_all=v_all.merge(v4.rename(columns={'avg':'avg_f1_run4'}),on='Classifier')
v_all.to_csv('all_avg_f1.csv')

In [15]:
v_all

,Classifier,avg_f1_run1,avg_f1_run2,avg_f1_run3,avg_f1_run4
0,Logistic Regression,0.1584,0.1584,0.1584,0.1584
1,K-Nearest Neighbors,0.2603,0.2603,0.2603,0.2603
2,Decision Tree,0.3289,0.3269,0.3282,0.3282
3,Random Forest,0.2820,0.2991,0.2869,0.2869
4,CNN,0.4976,0.4343,0.4648,0.4861
5,PyramidalCNN,0.5322,0.4957,0.5628,0.5195
6,LSTM,0.0000,0.0000,0.0000,0.0000
7,biLSTM,0.0000,0.0000,0.0000,0.0000
8,ConvLSTM,0.4994,0.4798,0.5093,0.3885
9,EEGNet,0.2652,0.0736,0.2251,0.1338
